# Table of Content
- [14.12 Debugging Basic Program Crashes](#14.12)
- [14.13 Profiling and Timing Your Program](#14.13)
- [14.14 Making Your Programs Run Faster](#14.14)

---
## <a name="14.12"></a> 14.12 Debugging Basic Program Crashes


- Start an interactive shell as soon as a program terminates
```sh
python3 -i yourprogram.py
```

- print traceback

In [1]:
import traceback
import sys

try:
    func(arg)
except Exception:
    traceback.print_exc(file=sys.stderr)

Traceback (most recent call last):
  File "<ipython-input-1-698f0c8e8b0e>", line 5, in <module>
    func(arg)
NameError: name 'func' is not defined


---
## <a name="14.13"></a> 14.13 Profiling and Timing Your Program

- Simply know the time of the whole program
```sh
time python3 yourprogram.py
```

- Get a detail report of the program
```sh
python3 -m cProfile yourprograms.py
```

- Studying the performance of small code fragments

In [2]:
from timeit import timeit

timeit('math.sqrt(2)', 'import math')

0.17056588000559714

In [3]:
timeit('sqrt(2)', 'from math import sqrt')

0.2299370560067473

---
## 14.14 Making Your Programs Run Faster

### Use Function
Code in global runs much slower than in functions due to the implementation of local versus global variables

In [4]:
%%time

a = 1
for _ in range(1000000):
    a += 1

CPU times: user 126 ms, sys: 1.42 ms, total: 127 ms
Wall time: 138 ms


In [5]:
%%time 
def func():
    a = 1
    for _ in range(1000000):
        a += 1

func()

CPU times: user 86.3 ms, sys: 5.37 ms, total: 91.7 ms
Wall time: 130 ms


### Selectively eliminate attribute Access
The use of dot(.) triggers methods such as `__getattribute__()` and `__getattr__()`, which often involves dict lockup

In [6]:
from timeit import timeit

timeit('math.sqrt(2)', 'import math')

0.15486120300192852

In [7]:
timeit('sqrt(2)', 'from math import sqrt')

0.10102694500528742

However, this only makes sense when the code is frequently executed. (e.g. loop)  
Otherwise, it might break readbility.

### Understand locality of variables

In [8]:
import math
from math import sqrt

def compute_roots_global_with_dot(nums):
    result = []
    for n in nums:
        result.append(math.sqrt(n))
    return result

def compute_roots_global(nums):
    result = []
    result_append = result.append
    for n in nums:
        result_append(sqrt(n))
    return result

def compute_roots_local(nums):
    local_sqrt = sqrt
    result = []
    result_append = result.append
    for n in nums:
        result_append(local_sqrt(n))
    return result

In [9]:
timeit('compute_roots_global_with_dot(range(10))', 'from __main__ import compute_roots_global_with_dot')

3.4750799049943453

In [10]:
timeit('compute_roots_global(range(10))', 'from __main__ import compute_roots_global')

2.7754836730018724

- The second is faster than the first one.
    - The use of sqrt instead of math.sqrt

In [11]:
timeit('compute_roots_local(range(10))', 'from __main__ import compute_roots_local')

2.6259513859986328


- The third one is even faster.
    - Assign math.sqrt as local variable

In general, ***looking up a value such as self.name will be slower than accessing a local variable***

### Avoid gratutious abstraction

In [12]:
class C:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, value): 
        self._y = value
        
c = C(1, 2)

timeit('c.x', 'from __main__ import c')

0.04473164399678353

In [13]:
timeit('c.y', 'from __main__ import c')

0.19272151798941195

The concept of using setter/getter is not necessary in Python.  
Using property when not needed will only slower the program

### Use the built-in containers

e.g. string, tuple, list, set, dict  
They are all implemented in C

### Other Discussion
- Before optimizaing, it's usually worthwhile to speedup the algorithms first.
- Don'y worry about optimization until you need to
    - John Ousterhout: "The best performance improvement is the transition from nonworking to the working state."

#### Why PyPy is faster?
PyPy analyzes the execution of the program and generates native machine code for frequently executed parts.